In [ ]:
import os
import prepro_options as opt
import preprocessor as prep
import postprocessor as postp
import processor as proc
import pandas as pd
import unifloc.uniflocpy.uTools.plotly_workflow as pw
import datetime
from sklearn import metrics
import multiprocessing as mp
from multiprocessing import Pool
import seaborn as sns

from sklearn import metrics
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.decomposition import PCA
from pandas.api.types import is_numeric_dtype

In [ ]:
well_name = 570
mark = ''
left_boundary = opt.get_left_bound(well_name)
right_boundary = opt.get_rigth_bound(well_name)

path_to_work_dir = f'data/{well_name}/restore_/'
chess_file_name = f'Скв. {well_name} ({opt.get_total_well_interval(well_name)}).xls'
cs_data_filename = f'data/{well_name}/init_edit/{well_name}_first_edit.csv'
create_input_data = True

# 5. Восстановление дебитов.
Нужно, используя сгенерированные данные `well_name_restore_input` восстановить дебиты с помощью `postprocessor.py`
Для этого активировать флаги
* vfm_calc_option = True
* restore_q_liq_only = True

и определить папку с входными данными `dir_name_with_input_data`

In [ ]:
dir_name_with_input_data = 'restore_input_' + '\\'
input_data_file_name = f'{well_name}_restore_input'
dir_name_with_calculated_data = 'restore_' + '\\'
calculated_data_file_name = f'{well_name}_restore_1'

In [ ]:
multiprocessing_on = True
if multiprocessing_on == True:
    filenames_list = []
    for (dirpath, dirnames, filenames) in os.walk(path_to_work_dir + 'multiprocessing\\'):
        filenames_list.extend(filenames)
        break
    print(filenames_list)
    for i,j in enumerate(filenames_list):
        if i == 0:
            first_result_data = pd.read_csv(path_to_work_dir + 'multiprocessing\\' + j , parse_dates = True, index_col = 'Время')
        else:
            another_result_data = pd.read_csv(path_to_work_dir + 'multiprocessing\\' + j , parse_dates = True, index_col = 'Время')
            first_result_data = first_result_data.append(another_result_data, sort = True)
            del first_result_data['d']
            first_result_data = first_result_data.dropna(subset = ['ESP.ESPpump.EffiencyESP_d'])
    first_result_data.to_csv(f'data/{well_name}/restore_/{well_name}_restore_1.csv')

Загрузка данных после восстановления дебитов

In [ ]:
calculated_data = prep.load_calculated_data_from_csv(f'data/{well_name}/restore_{mark}/{well_name}_restore_1.csv')

Объединение входных и выходных данным модели. Сохранение результатов в `well_name_calc_and_input.csv`

In [ ]:
input_data = pd.read_csv(f'data/{well_name}/restore_input/{well_name}_restore_input.csv',
                         parse_dates=True, index_col = 'Время')
all_data = input_data.join(calculated_data, how = 'outer')
all_data.to_csv(f'data/{well_name}/restore_/{well_name}_calc_and_input.csv')

In [ ]:
all_data.sort_index(inplace=True)

Построение графиков восстановления дебитов

In [ ]:
plt.scatter(all_data.index, all_data['Q ж, м3/сут (Модель)'].values - all_data['Объемный дебит жидкости (СУ)'].values)
plt.show()

# Так, теперь погнали избавимся от выбросов
Судя по всему фильтрация до запуска `processor.py` приводит к каким-то ошибкам

In [ ]:
plt.scatter(all_data['Объемный дебит жидкости (СУ)'], all_data['Время замера фактическое (СУ)'])
plt.xlabel('Дебит')
plt.ylabel('Время замера')
plt.title(f'Well {well_name}')
plt.show()
print(len(all_data['Объемный дебит жидкости (СУ)']))

Пока-что выставим ручками значение "ошибочного замера"

Здесь даже короткий замер не приводит к аномальным значениям дебита, но лучше бы его убрать.

In [ ]:
all_data = all_data[all_data['Время замера фактическое (СУ)'] >= 50]

Кроме того, очень важно, чтобы аномальные точки, как например `09.09.18` в скважине `570` обязательно попали на обучение. Таким образом система поймёт важность, как в описанном случае, `ГФ`

## Теперь настроим регрессию на половине данных - через одну точку, начиная с первой и потестим её на второй половине

Все данные нам не нужны, отбросим очевидно бесполезные ручками

In [ ]:
x = all_data[['ГФ (СУ)',
              'Процент обводненности (СУ)',
              'Давление на приеме насоса (пласт. жидкость) (СУ)',
              'Рбуф (Ш)', 'Температура на приеме насоса (пласт. жидкость) (СУ)',
              'F вращ ТМ (Ш)', 'Dшт (Ш)',
              'Активная мощность (СУ)',
              'Напряжение на выходе ТМПН (СУ)',
              'Коэффициент мощности (СУ)',
              'Объемный дебит жидкости (СУ)',
              'Q ж, м3/сут (Модель)',
              'F вращ ТМ (Ш)'
             ]
            ].copy()


print(f'number of NaN values: {np.isnan(x.values).sum()}/{x.shape[0]}')
x.dropna(inplace=True)
data = x.dropna()

y = pd.Series(x['Объемный дебит жидкости (СУ)'].values - x['Q ж, м3/сут (Модель)'].values)
x.drop(columns=['Объемный дебит жидкости (СУ)', 'Q ж, м3/сут (Модель)'], inplace=True)

In [ ]:
def get_test_train_drop_2_points(data: pd.DataFrame, target: pd.Series):
    out_x_train = data[data.index % 2 == 0]
    out_y_train = target[target.index % 2 == 0]
    out_x_test = data[data.index % 2 == 1]
    out_y_test = target[target.index % 2 == 1]
    return out_x_train, out_x_test, out_y_train, out_y_test

def get_joined_2_points_target(y_test, y_train):
    out = [None] * (len(y_test) + len(y_train))
    y_test = list(y_test)
    y_train = list(y_train)
    for i in range(len(out)):
        if i % 2 == 0:
            out[i] = y_train[int(i/2)]
        else:
            out[i] = y_test[int((i-1)/2)]
    return out

In [ ]:
date_col = x.index
x.reset_index(drop=True, inplace=True)

x_train, x_test, y_train, y_test = get_test_train_drop_2_points(x, y)

In [ ]:
y_test = y_test.values.reshape((-1, 1))
y_train = y_train.values.reshape((-1, 1))

sc_x = StandardScaler()
sc_x.fit(x_train)

sc_y = StandardScaler()
sc_y.fit(y_train)

x_train_sc = sc_x.transform(x_train)
x_test_sc = sc_x.transform(x_test)

y_train_sc = sc_y.transform(y_train)
y_test_sc = sc_y.transform(y_test)

In [ ]:
parameters = {'alpha': np.linspace(1e-7, 30, 1000)}

r_est = linear_model.Ridge()
gs_cv = GridSearchCV(r_est, parameters, cv=7)
_ = gs_cv.fit(x_train_sc, y_train_sc)
b_r_est = gs_cv.best_estimator_
predicted_err = sc_y.inverse_transform(b_r_est.predict(sc_x.transform(x)))

In [ ]:
plt.plot(date_col, predicted_err[:, 0] + data['Q ж, м3/сут (Модель)'].values, label='predicted')
plt.plot(date_col, data['Объемный дебит жидкости (СУ)'], label='y_true')
plt.legend(loc='upper right', bbox_to_anchor=(1.24, 0.8))
plt.xlabel('date')
plt.ylabel('q_liq, m3/day')
plt.show()

Вроде нормал, посчитаем `mae`

In [ ]:
err = metrics.mean_absolute_error(predicted_err[:, 0] + data["Q ж, м3/сут (Модель)"].values,
                                  data["Объемный дебит жидкости (СУ)"])
print(f'MAE_score: {err}')
print(f'MAE_score as %: {err/data["Объемный дебит жидкости (СУ)"].mean() * 100}')

Посмотрим на ошибки на обучении/тесте

In [ ]:
predicted_err_test = b_r_est.predict(x_test_sc)
print(f'MAE test,  scaled err = {metrics.mean_absolute_error(y_test_sc, predicted_err_test)}')
predicted_err_train = b_r_est.predict(x_train_sc)
print(f'MAE train, scaled err = {metrics.mean_absolute_error(y_train_sc, predicted_err_train)}')

# хммм, 15% это не так плохо
## посмотрим на "интегральную ошибку"

In [ ]:
def get_integrted_data(y: list, date, crit_time_delta=datetime.timedelta(2, 0 * 60 * 60*12 )):
    out = 0
    if len(date) != len(y):
        print('not the same length')
        return -1
    for i in range(len(date)-1):
        if date[i + 1]- date[i] <=  crit_time_delta:
            out += 0.5 * (y[i] + y[i+1])
        else:
            pass# print(f'long time no data, {date[i]} - {date[i+1]}')
    return out

In [ ]:
int_0 = get_integrted_data(predicted_err[:, 0] + data["Q ж, м3/сут (Модель)"].values, date_col)
int_err = int_0 - get_integrted_data( data["Объемный дебит жидкости (СУ)"].values, date_col)
print(f'integrated error = {abs(int_err)/int_0 * 100}%')

# Попробуем улучшить
Сейчас есть скачёк вниз, котрый, возможно, обусловленн `ГФ(СУ)`. А может это просто плохой замер

При этом, почти совсем нет реакции на эту точку.

Хотелось бы подобарть признак, который отловит этот скачок.

1. Посмотрим, что будет, если просто размножить эту точку

- Всё-таки посмотрим на корреляции и важность признаков

In [ ]:
m = all_data['Объемный дебит жидкости (СУ)'].mean()
sig = np.sqrt(all_data['Объемный дебит жидкости (СУ)'].var())
mh_train = all_data[np.abs(all_data['Объемный дебит жидкости (СУ)'] - m) > 3 * sig]

In [ ]:
mh_train

# Настакиваем аномальные точки

In [ ]:
mh_train = mh_train[['ГФ (СУ)', 'Процент обводненности (СУ)', 'Давление на приеме насоса (пласт. жидкость) (СУ)',
                  'Рбуф (Ш)', 'Температура на приеме насоса (пласт. жидкость) (СУ)', 'F вращ ТМ (Ш)', 'Dшт (Ш)',
                  'Активная мощность (СУ)', 'Напряжение на выходе ТМПН (СУ)', 'Коэффициент мощности (СУ)',
                  'Объемный дебит жидкости (СУ)', 'Q ж, м3/сут (Модель)', 'F вращ ТМ (Ш)']
                 ]

x_train_ap = mh_train.drop(columns=['Объемный дебит жидкости (СУ)', 'Q ж, м3/сут (Модель)'])
x_train_ap.reset_index(drop=True, inplace=True)

y_train_ap = pd.Series(mh_train['Объемный дебит жидкости (СУ)'].values - mh_train['Q ж, м3/сут (Модель)'].values)
y_train_ap.reset_index(drop=True, inplace=True)


for i in range(7):
    x_train = x_train.append(x_train_ap)
    y_train = np.append(y_train, y_train_ap.values.reshape((-1, 1)), axis=0)

In [ ]:
sc_x = StandardScaler()
sc_x.fit(x_train)

sc_y = StandardScaler()
sc_y.fit(y_train)

x_train_sc = sc_x.transform(x_train)
x_test_sc = sc_x.transform(x_test)

y_train_sc = sc_y.transform(y_train)
y_test_sc = sc_y.transform(y_test)

parameters = {'alpha': np.linspace(1e-7, 30, 1000)}

r_est = linear_model.Ridge()
gs_cv = GridSearchCV(r_est, parameters, cv=7)
_ = gs_cv.fit(x_train_sc, y_train_sc)
b_r_est = gs_cv.best_estimator_
predicted_err = sc_y.inverse_transform(b_r_est.predict(sc_x.transform(x)))

In [ ]:
plt.plot(date_col, predicted_err[:, 0] + data['Q ж, м3/сут (Модель)'].values, label='predicted')
plt.plot(date_col, data['Объемный дебит жидкости (СУ)'], label='y_true')
plt.legend(loc='upper right', bbox_to_anchor=(1.24, 0.8))
plt.xlabel('date')
plt.ylabel('q_liq, m3/day')
plt.show()

In [ ]:
err = metrics.mean_absolute_error(predicted_err[:, 0] + data["Q ж, м3/сут (Модель)"].values,
                                  data["Объемный дебит жидкости (СУ)"])
print(f'MAE_score: {err}')
print(f'MAE_score as %: {err/data["Объемный дебит жидкости (СУ)"].mean() * 100}')

Вроде бы `MAE` увеличилась, а ошибка за интерграл - уменьшилась

In [ ]:
x_train, x_test, y_train, y_test = get_test_train_drop_2_points(x, y)

y_test = y_test.values.reshape((-1, 1))
y_train = y_train.values.reshape((-1, 1))

sc_x = StandardScaler()
sc_x.fit(x_train)

sc_y = StandardScaler()
sc_y.fit(y_train)

x_train_sc = sc_x.transform(x_train)
x_test_sc = sc_x.transform(x_test)

y_train_sc = sc_y.transform(y_train)
y_test_sc = sc_y.transform(y_test)

predicted_err = sc_y.inverse_transform(b_r_est.predict(sc_x.transform(x)))

int_0 = get_integrted_data(predicted_err[:, 0] + data["Q ж, м3/сут (Модель)"].values, date_col)
int_err = int_0 - get_integrted_data( data["Объемный дебит жидкости (СУ)"].values, date_col)
print(f'integrated error = {abs(int_err)/int_0 * 100}%')

# Всё-таки посмотрим на корреляции и важность признаков

In [ ]:
x_corr = all_data[['ГФ (СУ)', 'Процент обводненности (СУ)', 'Давление на приеме насоса (пласт. жидкость) (СУ)',
                   'Рбуф (Ш)', 'Температура на приеме насоса (пласт. жидкость) (СУ)', 'F вращ ТМ (Ш)', 'Dшт (Ш)',
                   'Активная мощность (СУ)', 'Напряжение на выходе ТМПН (СУ)', 'Коэффициент мощности (СУ)',
                   'Объемный дебит жидкости (СУ)', 'F вращ ТМ (Ш)', 'Q ж, м3/сут (Модель)'
                  ]].dropna()
x_corr['ERROR'] = x_corr['Объемный дебит жидкости (СУ)'].values - x_corr['Q ж, м3/сут (Модель)'].values
corr = x_corr.drop(columns=['Объемный дебит жидкости (СУ)', 'Q ж, м3/сут (Модель)', 'Dшт (Ш)']).corr()

mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

Кажется, линейной связи между `ГФ` и ошибкой нет. Чтож, видимо она не линейная.

Вообще, судя по графикам, нас волнуют только лишь значительые изменения `ГФ`, мб попробовать перейти к `ln(ГФ)`?

In [ ]:
x = all_data[['ГФ (СУ)',
              'Процент обводненности (СУ)',
              'Давление на приеме насоса (пласт. жидкость) (СУ)',
              'Рбуф (Ш)', 'Температура на приеме насоса (пласт. жидкость) (СУ)',
              'F вращ ТМ (Ш)', 'Dшт (Ш)',
              'Активная мощность (СУ)',
              'Напряжение на выходе ТМПН (СУ)',
              'Коэффициент мощности (СУ)',
              'Объемный дебит жидкости (СУ)',
              'Q ж, м3/сут (Модель)',
              'F вращ ТМ (Ш)'
             ]
            ].copy()


print(f'number of NaN values: {np.isnan(x.values).sum()}/{x.shape[0]}')
x.dropna(inplace=True)
data = x.dropna()
    
x['ГФ (СУ)'] = np.log(x['ГФ (СУ)'].values)
x.replace([np.inf, -np.inf], np.nan, inplace=True)
x.dropna(inplace=True)    
y = pd.Series(x['Объемный дебит жидкости (СУ)'].values - x['Q ж, м3/сут (Модель)'].values)
y_0 = x['Q ж, м3/сут (Модель)']
y_0_true= x['Объемный дебит жидкости (СУ)']
x.drop(columns=['Объемный дебит жидкости (СУ)', 'Q ж, м3/сут (Модель)'], inplace=True)
date_col = x.index

In [ ]:
x.reset_index(drop=True, inplace=True)

x_train, x_test, y_train, y_test = get_test_train_drop_2_points(x, y)

sc_x = StandardScaler()
sc_x.fit(x_train)

sc_y = StandardScaler()
sc_y.fit(y_train.values.reshape((-1, 1)))

x_train_sc = sc_x.transform(x_train)
x_test_sc = sc_x.transform(x_test)

y_train_sc = sc_y.transform(y_train.values.reshape((-1, 1)))
y_test_sc = sc_y.transform(y_test.values.reshape((-1, 1)))

parameters = {'alpha': np.linspace(1e-7, 30, 1000)}

r_est = linear_model.Ridge()
gs_cv = GridSearchCV(r_est, parameters, cv=7)
_ = gs_cv.fit(x_train_sc, y_train_sc)
b_r_est = gs_cv.best_estimator_
predicted_err = sc_y.inverse_transform(b_r_est.predict(sc_x.transform(x)))

In [ ]:
plt.plot(date_col, y_0_true, label='y_true')
plt.plot(date_col, predicted_err[:, 0] + y_0.values, label='predicted')
plt.legend(loc='upper right', bbox_to_anchor=(1.24, 0.8))
plt.xlabel('date')
plt.ylabel('q_liq, m3/day')
plt.show()

In [ ]:
err = metrics.mean_absolute_error(predicted_err[:, 0] + y_0.values,
                                  y_0_true)
print(f'MAE_score: {err}')
print(f'MAE_score as %: {err/y_0_true.mean() * 100}')

In [ ]:
x_train, x_test, y_train, y_test = get_test_train_drop_2_points(x, y)

y_test = y_test.values.reshape((-1, 1))
y_train = y_train.values.reshape((-1, 1))

sc_x = StandardScaler()
sc_x.fit(x_train)

sc_y = StandardScaler()
sc_y.fit(y_train)

x_train_sc = sc_x.transform(x_train)
x_test_sc = sc_x.transform(x_test)

y_train_sc = sc_y.transform(y_train)
y_test_sc = sc_y.transform(y_test)

predicted_err = sc_y.inverse_transform(b_r_est.predict(sc_x.transform(x)))

int_0 = get_integrted_data(predicted_err[:, 0] + y_0.values, date_col)
int_err = int_0 - get_integrted_data(y_0_true, date_col)
print(f'integrated error = {abs(int_err)/int_0 * 100}%')

Опять, видимо 2 метрики `MAE` и ошибка по интегралу - довольно разные

Выглядит просто каеф. Например, в скважине `693` есть несинхронные пики, ну а интеграл не сильно портится от того что они 
не синхронные

Наверное всё-таки нужно построить что-то типа `_report`, попробуемсс

In [ ]:
overall_data = all_data.dropna()
overall_data['Q_l(ML error predicted)'] = predicted_err[:, 0] + data["Q ж, м3/сут (Модель)"].values
overall_data['Q_l error unifloc'] = overall_data['Объемный дебит жидкости (СУ)'].values-overall_data['Q ж, м3/сут (Модель)'].values
overall_data['Q_l error (ML restored)'] = predicted_err[:, 0]
overall_data.rename(columns={"Q ж, м3/сут (Модель)": 'Q_l pred unifloc(calibr = 1)', 
                             'Объемный дебит жидкости (СУ)': 'Q_l true'
                            }, inplace=True)

In [ ]:
errors = {'Errors': ['Q_l error unifloc', 'Q_l error (ML restored)']}

liquid_rates = {'Дебиты':  ['Q_l true', 'Q_l pred unifloc(calibr = 1)', 'Q_l(ML error predicted)']}

essential_metrics = {'Обводнённость и ГФ': ['Процент обводненности (СУ)', 'ГФ (СУ)']}

pressures = {'Давления': ['Давление на приеме насоса (пласт. жидкость) (СУ)','Рбуф (Ш)',
             'Линейное давление (СУ)']}

temperatures = {'Температуры': ['Температура на приеме насоса (пласт. жидкость) (СУ)', 'T прием ЭЦН, C (Модель)',
                'T поверхности, С (Модель)', 'T выкид ЭЦН, С (Модель)', 'T прием, С  (Модель)', 'T устья, С (Модель)']
               }

power = {'Мощность': ['Активная мощность (СУ)', 'Мощность, передаваемая ЭЦН (Модель)',
         'Мощность, передаваемая жидкости (Модель)','Мощность, передаваемая кабелю (Модель)', 
         'Мощность, потребляемая СУ (Модель)', 'Мощность, потребляемая газосепаратором (Модель)', 
         'Мощность, потребляемая протектором (Модель)', 'Мощность, потребляемая ТМПН (Модель)', 
         'Мощность, передаваемая СУ (Модель)', 'Мощность, передаваемая ПЭД (Модель)']
        }

load = {'Загрузка': ['Загрузка двигателя (Модель)']}

all_banches = [liquid_rates, errors, essential_metrics, pressures, temperatures, power, load]